<a href="https://colab.research.google.com/github/jaswanthakula-in/Fraud-detection-using-ieee-dataset/blob/main/QLoraLNNTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft

In [5]:
!pip uninstall -y bitsandbytes
!pip uninstall -y transformers accelerate peft

Found existing installation: bitsandbytes 0.45.4
Uninstalling bitsandbytes-0.45.4:
  Successfully uninstalled bitsandbytes-0.45.4
Found existing installation: transformers 4.51.0
Uninstalling transformers-4.51.0:
  Successfully uninstalled transformers-4.51.0
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
Found existing installation: peft 0.15.1
Uninstalling peft-0.15.1:
  Successfully uninstalled peft-0.15.1


In [6]:
!pip install bitsandbytes --prefer-binary
!pip install -U transformers accelerate peft

  Using cached bitsandbytes-0.45.4-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
Using cached bitsandbytes-0.45.4-py3-none-manylinux_2_24_x86_64.whl (76.0 MB)
  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.15.1-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.51.0-py3-none-any.whl (10.4 MB)
Using cached accelerate-1.6.0-py3-none-any.whl (354 kB)
Using cached peft-0.15.1-py3-none-any.whl (411 kB)


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
from peft import get_peft_model, LoraConfig, TaskType

# Configure LoRA for GPT-2
peft_config = LoraConfig(
    r=8,                          # low-rank dimension
    lora_alpha=32,               # scaling factor
    target_modules=["c_attn", "c_proj"],  # GPT-2 specific attention projection layers
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Inject LoRA adapters into the model
model = get_peft_model(model, peft_config)

# Optional: Show trainable parameters
model.print_trainable_parameters()

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [7]:
from datasets import load_dataset

# 1. Load small sample of Wikitext
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")

# 2. FIX: Assign a pad token for GPT-2 tokenizer
tokenizer.pad_token = tokenizer.eos_token

# 3. Tokenize
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# 4. Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/367 [00:00<?, ? examples/s]

In [10]:
from torch.utils.data import DataLoader
import torch

# Format dataset for PyTorch
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Create DataLoader
train_loader = DataLoader(tokenized_dataset, batch_size=4, shuffle=True)

In [15]:
!pip install -q codecarbon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.7/516.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.9.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [18]:
# ✅ Step 6 — START of training
from codecarbon import EmissionsTracker
import time

start_time = time.time()
tracker = EmissionsTracker(project_name="hybrid_model", log_level="error")
tracker.start()

# ✅ Training loop
from torch.optim import AdamW
from tqdm import tqdm

model.train()
optimizer = AdamW(model.parameters(), lr=2e-4)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for epoch in range(1):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        inputs = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=inputs)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

# ✅ Step 6 — END of training
end_time = time.time()
tracker.stop()

Epoch 0: 100%|██████████| 92/92 [00:06<00:00, 14.77it/s, loss=0.14]


2.75983906118117e-05

In [19]:
model.eval()

prompt = "The future of energy-efficient AI is"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

print("\n🧠 Generated Output:\n")
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🧠 Generated Output:

The future of energy-efficient AI is in the spotlight , , in particular , with , . . . , , . . , . , . , . , . , . , . ,


In [20]:
import torch
import os

# ✅ Final loss from training loop
print(f"\n📉 Final Loss: {loss.item():.4f}")

# ✅ Perplexity
perplexity = torch.exp(torch.tensor(loss.item()))
print(f"🤯 Perplexity: {perplexity:.2f}")

# ✅ Training time
print(f"⏱️ Training Time: {end_time - start_time:.2f} seconds")

# ✅ GPU memory usage
print("💾 GPU Memory Usage:")
!nvidia-smi --query-gpu=memory.used,memory.total --format=csv,noheader,nounits

# ✅ Energy consumed (from codecarbon emissions.csv)
emissions_file = "emissions.csv"
if os.path.exists(emissions_file):
    with open(emissions_file, "r") as f:
        lines = f.readlines()
        if len(lines) > 1:
            last_line = lines[-1].split(",")
            energy = float(last_line[6])  # 7th column is emissions (kgCO₂e)
            print(f"🔋 Energy Consumed: {energy:.6f} kgCO₂e")
        else:
            print("⚠️ No emissions data found.")
else:
    print("⚠️ emissions.csv file not found. Was CodeCarbon tracker used?")


📉 Final Loss: 0.1398
🤯 Perplexity: 1.15
⏱️ Training Time: 7.73 seconds
💾 GPU Memory Usage:
1364, 15360
🔋 Energy Consumed: 0.000004 kgCO₂e
